In [3]:
import os 
import pandas as pd
import numpy as np

In [5]:
os.chdir('S:/Spring/NLP')
os.listdir()

['Assignment 1 - Facebook Post Classification.ipynb',
 'FB_posts_labeled.txt',
 'FB_posts_unlabeled.txt',
 'Text Preprocessing and Representation Learning.html']

In [6]:
# df =  pd.read_csv('')

posts = []
for line in open("FB_posts_labeled.txt"):
    if 'postId' not in line:
        pid, message, Appreciation, Complaint, Feedback = line.rstrip('\n').split('\t')
        posts.append(message)
print(len(posts))

7961


In [27]:
# convert a tab delimited file to data frame

df_post_label = pd.DataFrame(columns=['pid', 'message', 'Appreciation', 'Complaint', 'Feedback'])
# df_post_label.columns = ['pid', 'message', 'Appreciation', 'Complaint', 'Feedback']

for line in open("FB_posts_labeled.txt"):
    if 'postId' not in line:
        pid, message, Appreciation, Complaint, Feedback = line.rstrip('\n').split('\t')
        
#         df_post_label = df_post_label.append(pd.DataFrame(pid, message, Appreciation, Complaint, Feedback))
        df_post_label.loc[len(df_post_label)] = [pid, message, Appreciation, Complaint, Feedback]

        

# df_post_label.columns = ['pid', 'message', 'Appreciation', 'Complaint', 'Feedback']
# df_post_label.shape

In [33]:
# df_post_label.head(10)
df_post_label[['Appreciation', 'Complaint', 'Feedback']].value_counts()

Appreciation  Complaint  Feedback
0             1          0           4255
1             0          0           2062
0             0          1           1644
dtype: int64

In [39]:

label_cols = ['Appreciation', 'Complaint', 'Feedback']
df_post_label[label_cols] = df_post_label[label_cols].apply(pd.to_numeric, axis = 1, errors = 'coerce')

df_post_label['label']= df_post_label[['Appreciation', 'Complaint', 'Feedback']].idxmax(axis = 1)
df_post_label.head()

,pid,message,Appreciation,Complaint,Feedback,label
0,126016648090_10150802142013091,Great ! ;),1,0,0,Appreciation
1,108381603303_10151136215833304,YUM! YUM!,1,0,0,Appreciation
2,108381603303_3913438087739,Yummm :)),1,0,0,Appreciation
3,110455108974424_343049739048292,sweet,1,0,0,Appreciation
4,110455108974424_350358541650745,nice,1,0,0,Appreciation


In [42]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_post_label['label_encoded'] = le.fit_transform(df_post_label['label'])
# df_post_label['label_encoded'].value_counts()

1    4255
0    2062
2    1644
Name: label_encoded, dtype: int64

# Text Pre-processing

### Emojis, emoticons, hashtags, to lowercase, newlines, multille spaces, non-english

In [44]:
!pip install emoji

     -------------------------------------- 240.9/240.9 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234927 sha256=f3f203f335f44d7bec53fd345ddd844189dcd3b2a7ecad95afd9e2a915886b77
  Stored in directory: c:\users\ranga\appdata\local\pip\cache\wheels\02\3d\88\51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji



[notice] A new release of pip available: 22.2.1 -> 23.0.1
[notice] To update, run: C:\Users\ranga\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [45]:

import re, string
import emoji
import nltk


df = df_post_label[['message', 'label']]
df.head()



,message,label
0,Great ! ;),Appreciation
1,YUM! YUM!,Appreciation
2,Yummm :)),Appreciation
3,sweet,Appreciation
4,nice,Appreciation


In [63]:
# def strip_emoji(text):
#     return re.sub(emoji.get_emoji_regexp(), r"", text)  #remove emoji

def strip_emoji(data):
    emoji_clean= re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    data=emoji_clean.sub(r'',data)
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)


def remove_special_characters(text):
    # Define the pattern to match special characters
    pattern = r'[^a-zA-Z0-9\s]'

    # Remove special characters using regular expression
    text = re.sub(pattern, '', text)

    return text

In [71]:
s = df.message[0]
s =  s+'"'
s
remove_special_characters(s).strip()
# s = s.decode('utf-8')


'Great'

In [75]:
# messages_new = []
# for t in df.message:
#     messages_new.append(remove_mult_spaces(strip_all_entities(remove_special_characters(t))))
    
# #     messages_new.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))
    
# df['message_clean'] = messages_new
df.tail(30)

,message,label,message_clean
7931,I will never fly US AIRWAYS again the check in...,Complaint,i will never fly us airways again the check in...
7932,My favorate financial institution!,Appreciation,my favorate financial institution
7933,"""Did you know? Most processed and package...",Complaint,did you know most processed and packaged foods...
7934,"""hola como estan usted y su hijo , espero q es...",Complaint,hola como estan usted y su hijo espero q esten...
7935,Dicks is a horrible place to buy anything from...,Complaint,dicks is a horrible place to buy anything from...
7936,Discover was really helpful whgen my Dad was d...,Complaint,discover was really helpful whgen my dad was d...
7937,So i send in 2 years tax retiurns over 10 pay ...,Complaint,so i send in 2 years tax retiurns over 10 pay ...
7938,i went to dicks on Monday evening to look at a...,Complaint,i went to dicks on monday evening to look at a...
7939,"""walmart in Twin Falls Idaho sucks i was there...",Complaint,walmart in twin falls idaho sucks i was there ...
7940,I don't know if this is the right place to do ...,Complaint,i dont know if this is the right place to do t...


In [80]:
import unicodedata

def remove_non_english(text):
    # Convert text to ASCII format
    ascii_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Check if text is in English language
    is_english = ascii_text.isascii()

    if not is_english:
        return text
    
    
for t in df.message_clean:
    print(remove_non_english(t))


None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
